# Classificador de imagens "*Thumbs Up*" e "*Thumbs Down*"
Autor: Bernardo Cassimiro Fonseca de Oliveira

Essa ferramenta classifica imagens com "*Thumbs Up*" e "*Thumbs Down*" utilizando uma [Máquina de Vetor Suporte](https://scikit-learn.org/stable/modules/svm.html) (SVM, do inglês *Support Vector Machine*) com pré-processamento usando uma [Histograma de Gradientes](https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_hog.html) (HOG, do inglês *Histogram of Gradients*).

O *dataset* utilizado para o treinamento desse classificador foi criado por mim mesmo, filmando minhas mãos em diferentes ângulos com o celular. Os *frames* desses vídeos foram extraídos, gerando 1400 imagens para o treinamento e 800 imagens para teste. Além disso, 51 imagens retiradas da internet foram utilizadas para testar a capacidade de extrapolação do classificador.

In [ ]:
# -*- coding: latin-1 -*-

# Importando bibliotecas -------------------------------------------------------
import cv2                        # Importa a OpenCV
import numpy as np                # Importa a NumPy
import os                         # Importa a Operational System
from sklearn import svm           # Importa a SVM da Scikit Learn
from skimage.feature import hog   # Importa o HOG da Scikit Image
from skimage import exposure      # Importa a Exposure da Scikit Image

**Carregamento das imagens**

Nesta seção, as imagens são carregadas, seu HOG é calculado, linearizado e armazenado em *arrays* que serão utilizadas no treinamento, teste e extrapolação da SVM.

O HOG foi aplicado nesse situação pois como o *dataset* de treino tem pouca variabilidade. Sem o HOG, a extrapolação teria um resultado ruim, pois o classificador jamais teria visto imagens similares. Com o HOG, um padrão no histogram emerge na presença de um "*Thumbs Up*" e de um "*Thumbs Down*", permitindo que, mesmo com imagens nunca vista, o classificador ainda consiga identificar o que precisa.

Os parâmetros do HOG foram ajustados para serem plausíveis considerando o tamanho da imagem de treino, cuja resolução espacial é igual a (32 x 24) pixels. É possível que o resultado da classificação pudesse ser ainda melhor ao se testar diferentes parâmetros, utilizando por exemplo uma estrutura como uma *random search* ou uma *grid search*.

In [ ]:
# Lendo as imagens de treinamento ----------------------------------------------
dirTRAINING = 'training'                                                        # Seta a pasta em que estão as imagens para o treinamento
listTRAINING = os.listdir(dirTRAINING)                                          # Lista a pasta em que estão as imagens para o treinamento
numTRAINING = len(listTRAINING)                                                 # Obtém o número de imagens de treinamento

training = np.zeros((numTRAINING,24*32)).astype(np.float64)                     # Inicializa um array 1400 x 768 de imagens linearizadas
for img in range(1,numTRAINING):
    imageNAME = ''.join([dirTRAINING,'\\',listTRAINING[img]])                   # Determina o endereço de cada imagem
    image = cv2.imread(imageNAME,0)                                             # Abre a imagem em tons de cinza
    _, hog_image = hog(image, orientations=18, pixels_per_cell=(4,4),           # Calcula o HOG da imagem
                    cells_per_block=(1,1), visualize=True, multichannel=False)
    image = exposure.rescale_intensity(hog_image, in_range=(0, 255))            # Impõe os limites da imagem do HOG entre 0 e 255 níveis de cinza
    imageLINE = image.reshape(-1,24*32).astype(np.float64)                      # Lineariza a imagem
    training[img,:] = imageLINE                                                 # Coloca a imagem linearizada no array de treinamento

In [ ]:
# Lendo as imagens de teste ----------------------------------------------------
dirTEST = 'test'                                                                # Seta a pasta em que estão as imagens para o teste
listTEST = os.listdir(dirTEST)                                                  # Lista a pasta em que estão as imagens para o teste
numTEST = len(listTEST)                                                         # Obtém o número de imagens de teste
    
test = np.zeros((numTEST,24*32)).astype(np.float64)                             # Inicializa um array 800 x 768 de imagens linearizadas
for img in range(1,numTEST):
    imageNAME = ''.join([dirTEST,'\\',listTEST[img]])                           # Determina o endereço de cada imagem
    image = cv2.imread(imageNAME,0)                                             # Abre a imagem em tons de cinza
    _, hog_image = hog(image, orientations=18, pixels_per_cell=(4,4),           # Calcula o HOG da imagem
                    cells_per_block=(1,1), visualize=True, multichannel=False)
    image = exposure.rescale_intensity(hog_image, in_range=(0, 255))            # Impõe os limites da imagem do HOG entre 0 e 255 níveis de cinza 
    imageLINE = image.reshape(-1,24*32).astype(np.float64)                      # Lineariza a imagem
    test[img,:] = imageLINE                                                     # Coloca a imagem linearizada no array de teste

In [ ]:
# Lendo as imagens para a extrapolação -----------------------------------------
dirGENERALIZATION = 'generalization'                                            # Seta a pasta em que estão as imagens para a extrapolação
listGENERALIZATION = os.listdir(dirGENERALIZATION)                              # Lista a pasta em que estão as imagens para a extrapolação
numGENERALIZATION = len(listGENERALIZATION)                                     # Obtém o número de imagens da extrapolação

generalization = np.zeros((numGENERALIZATION,24*32)).astype(np.float64)         # Inicializa um array 51 x 768 de imagens linearizadas
for img in range(1,numGENERALIZATION):
    imageNAME = ''.join([dirGENERALIZATION,'\\',listGENERALIZATION[img]])       # Determina o endereço de cada imagem
    image = cv2.imread(imageNAME,0)                                             # Abre a imagem em tons de cinza
    _, hog_image = hog(image, orientations=18, pixels_per_cell=(4,4),           # Calcula o HOG da imagem
                    cells_per_block=(1,1), visualize=True, multichannel=False)
    image = exposure.rescale_intensity(hog_image, in_range=(0, 255))            # Impõe os limites da imagem do HOG entre 0 e 255 níveis de cinza 
    imageLINE = image.reshape(-1,24*32).astype(np.float64)                      # Lineariza a imagem
    generalization[img,:] = imageLINE                                           # Coloca a imagem linearizada no array da extrapolação

**Criação das *labels* dos *datasets***

Nesta seção, são criadas as *labels* dos *datasets*. Por conveniência, os *datasets* de treinamento e do teste foram organizados de forma que todas as imagens *Thumbs Up* vem primeiro no array depois vem todas as imagens *Thumbs Down*. Apenas o *dataset* da extrapolação tem uma ordem diferenciada.

A *label* "1" identifica um "*Thumbs Up*" e a *label* "0" identifica um "*Thumbs Down*".

In [ ]:
# Criando as labels ------------------------------------------------------------
# trainingCSV = 'trainingLABELS.csv'
# trainingLABELS = np.genfromtxt(trainingCSV, delimiter = '')
trainingLABELS = np.concatenate((np.ones(int(numTRAINING/2)),                   # Gera um array de labels para o treinamento
                                 np.zeros(int(numTRAINING/2))))

# testCSV = 'testLABELS.csv'
# testLABELS = np.genfromtxt(testCSV, delimiter = '')
testLABELS = np.concatenate((np.ones(int(numTEST/2)),                           # Gera um array de labels para o teste
                             np.zeros(int(numTEST/2))))

# generalizationCSV = 'generalizationLABELS.csv'
# generalizationLABELS = np.genfromtxt(generalizationCSV, delimiter = '') 
generalizationLABELS = np.array([0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,    # Gera um array de labels para a extrapolação
                                 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1.,
                                 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1.,
                                 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
                                 1., 1., 1., 1., 0., 1., 1.])

**Treinamento da SVM**

Nesta seção, é realizado o treinamento da SVM, utilizando os parâmetros padrão da ferramenta. Também não foram testados aqui diferentes parâmetros, de forma ser possível que o resultado da classificação possa ainda ser aprimorado.

A SVM foi escolhida para essa classificação por ser uma ferramenta bastante conhecida, capaz de prover bons resultados de classificação, e que não exige um grande *dataset* para o treinamento.

In [ ]:
# Treinando a SVM --------------------------------------------------------------
clf = svm.SVC()                     # Inicializa a ferramenta
clf.fit(training, trainingLABELS)   # Realiza o treinamento

**Verificação da performance da ferramenta com as imagens de teste e de extrapolação**

Por fim, a performance do classificador é avaliado num conjunto de teste e num de extrapolação utilizadno a acurácia como métrica. Outras métricas podem ser utilizadas para avaliar diferentes aspectos do desempenho da ferramenta, como o recall, o F-score e o coeficiente de correlação de Matthews.

In [ ]:
# Verificando a acurácia do teste ----------------------------------------------
result = clf.predict(test)

matches = result == testLABELS
correct = np.count_nonzero(matches)
accuracy = correct*100.0/len(result)
print(accuracy)

# Verificando a acurácia da generalização --------------------------------------
resultGEN = clf.predict(generalization)

matchesGEN = resultGEN == generalizationLABELS
correctGEN = np.count_nonzero(matchesGEN)
accuracyGEN = correctGEN*100.0/len(resultGEN)
print(accuracyGEN)

Vê-se que a acurácia no conjunto de teste, com imagens similares às que foram usadas no treinamento, foi de {{accuracy}} %. O resultado foi bastante bom, mas que também pode indicar um overfitting da ferramenta.

Como a acurácia no conjunto de extrapolação, com imagens bem diferentes em relação às que foram usadas no treinamento, foi de {{accuracyGEN}} %, pode se dizer que o classificador é capaz de generalizar, pois o resultado também foi alto, apesar de um pouco menor que o de teste.

Conforme abordado anteriormente, o resultado da classificação pode vir a ser aprimorado ao se testar sistematicamente outros parâmetros do HOG e da SVM, bem como ao adicionar outras ferramentas de pré-processamento como a binzarização das imagens com a [segmentação de Otsu](https://ieeexplore.ieee.org/document/4310076) ou até mesmo a aplicação de outra ferramenta inteligente como a segmentação de instância com uma [Mask R-CNN](https://arxiv.org/abs/1703.06870) ou uma [Unet](https://arxiv.org/abs/1505.04597). 